In [ ]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import logging
import difflib
import Levenshtein as lev
from tqdm.notebook import tqdm
import multiprocessing
from gensim.models import Doc2Vec
import gensim.models.doc2vec
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import spatial
import re
import movies_df_helper_functions as helper_fn
import random

%matplotlib inline

In [ ]:
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

In [ ]:

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tqdm.pandas(desc="my bar!")

In [ ]:
# load movies.csv

movies_file_df = pd.read_csv('generated_dataset/final_lod_movies_dataset.csv')

In [ ]:
movies_file_df.describe

In [ ]:
movies_file_df.shape

In [ ]:
movies_file_df.info()

In [ ]:
movies_file_df.describe(include=['object', 'bool'])

In [ ]:
movies_file_df

***********************Prepare the movies information to be used for training the doc2vec model ***********************
\rnsteps:
Step 1-split actor_name, director_name,writer_name,genere_id,country_id and convert it to list of words
#Step 2-concatenate first and last names with an _
#Step 3.convert all data to lower case
#Step 4-concatenate movie_id and title to overcome the issue of movies having the same title
#Step 5-create a sentence to represent each movie by combining all attributes for every movie in one string with space separating each value


In [ ]:
#copy data into new dataframe
movies_df = movies_file_df[['movie_id','title','actor_name','director_name','writer_name','genre','country']].copy()

In [ ]:
movies_df[7770:7778]

In [ ]:
movies_df.describe

In [ ]:
def data_processing(df, col_name):    
    regex = "\(.*?\)"
    df[col_name] = df[col_name].apply(lambda row: re.sub(regex,'',row) if row is not None else row)
    df[col_name] = df[col_name].apply(lambda row: row.lower().strip().replace(' ','_').split('|') if row is not None else row)
    df[col_name] = df[col_name].apply(lambda row: [item.strip() for item in row] if row is not None else row)
    return df

In [ ]:

movies_df = data_processing(movies_df, 'actor_name')
movies_df = data_processing(movies_df, 'writer_name')
movies_df = data_processing(movies_df, 'genre')
movies_df = data_processing(movies_df, 'country')
movies_df = data_processing(movies_df, 'director_name')
movies_df.describe

In [ ]:
movies_df[9000:9030].tail()

In [ ]:
movies_df['movie_id']=movies_df['movie_id'].str.lower()
movies_df['title']=movies_df['title'].str.lower()
movies_df[['movie_id','title']]

In [ ]:
movies_df['movie_sentence'] = movies_df.progress_apply(lambda row:  fn.build_sentence_from_col(row.actor_name, row.genre, row.director_name, row.writer_name, row.country), axis=1)

In [ ]:
#check if any movie has empty sentence
movies_df[movies_df['movie_sentence'].str.len() == 0].count()

In [ ]:
movies_df['movie_sentence']

In [ ]:
movies_df[18:]

In [ ]:
movies_df.to_csv('generated_dataset/lod_doc2vec_train_data.csv')

# ********************************* Create the doc2vec model **********************

In [ ]:
# Create the tagged document needed for Doc2Vec
tagged_documents=[]
movies_df.progress_apply(lambda row : tagged_documents.append(TaggedDocument(row.movie_sentence,[row.movie_id])),axis=1)

#convert the series of tagged docuemnt to a list
train_data = list(tagged_documents)

print("Length of training data {}".format(len(train_data)))

In [ ]:
train_data

In [ ]:
random.seed = 400
[random.shuffle(data.words) for data in train_data]

In [ ]:
train_data

In [ ]:

#Configure and train the model
model = gensim.models.doc2vec.Doc2Vec(dm= 0, vector_size= 70, sample= 0, window= 5, min_count=1, workers=cores, epochs= 100)
model.build_vocab(train_data)
model.train(train_data, total_examples=len(train_data) , epochs=model.epochs)
#save the model
model.save("models/exp4/model")